In [18]:
import numpy as np
import networkx as nx
import matplotlib.patches as mpatches
from matplotlib import colors
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
import pickle as pk

In [19]:
# district number
DISTRICT_NUM = 8
# county width
ASSIGN_W = 5
# county height
ASSIGN_H = 8

popBound = 0.05


# creat district set
district = []
districtSet = []
for idx in range(DISTRICT_NUM):
    district.append("d"+str(idx+1))
    districtSet.append("d"+str(idx+1))
nodes = []
for idx in range(ASSIGN_W):
    for jdx in range(ASSIGN_H):
        nodes.append("n_x{}_y{}".format(idx,jdx))

In [20]:
# construct adjacency data
adjList = []
for i in range(ASSIGN_W):
    for j in range(ASSIGN_H):
        # right adj
        if i + 1 < ASSIGN_W:
            adjList.append(["n_x{}_y{}".format(i,j),"n_x{}_y{}".format(i+1,j)])
        # up adj
        if j + 1 < ASSIGN_H:
            adjList.append(["n_x{}_y{}".format(i,j),"n_x{}_y{}".format(i,j+1)])
            
# construct node position
position = {}
population = {}
totalPop = 0
unitNumber = {} #used to count the units in it
for idx in range(ASSIGN_W):
    for jdx in range(ASSIGN_H):
        position["n_x{}_y{}".format(idx,jdx)] = (0.5+1*idx, 0.5+1*jdx)

        population["n_x{}_y{}".format(idx,jdx)] = random.randrange(10,300)
            
        totalPop += population["n_x{}_y{}".format(idx,jdx)]
        unitNumber["n_x{}_y{}".format(idx,jdx)] = 1
        
interval = 2*(totalPop/DISTRICT_NUM)*popBound
        

Construct initial Graph

In [21]:
with open("adj_5_8.pk","wb") as f:
    pk.dump(adjList,f)
with open("population_5_8.pk","wb") as f:
    pk.dump(population,f)
with open("position_5_8.pk","wb") as f:
    pk.dump(position,f)

In [424]:
def merge(G, nodeA, nodeB):
     # Get all adjacent nodes of two nodes
    assert (nodeA in G.nodes) and (nodeB in G.nodes), "Input node(s) does not exist!"
    new_adj = []
    new_adj += list(G[nodeA])
    # check if node A and node B are adjacent
    assert (nodeB in new_adj) , "Input nodes are not adjacent!"

    new_adj += G[nodeB]
    # Create the new node with combined name
    name = str(nodeA) + '/' + str(nodeB)
    # Add new edges
    G.add_edges_from([(p, name) for p in new_adj])
    # Remove old nodes and the edges related to old nodes will also be removed
    G.remove_nodes_from([nodeA, nodeB])
    
    return name

In [425]:
def EuclieanDistance(A,B):
    return ((A[0]-B[0])**2 + (A[1]-B[1])**2)**(1/2)

In [426]:
'''
G = nx.Graph()
G.add_edges_from(adjList)
# in order to make d districts, we need to run merge for n-d times
originalNodeNum = len(G.nodes)
for i in range(originalNodeNum-DISTRICT_NUM):
    # randomly choose one node
    currentNodeNum = originalNodeNum - i
    idx = random.randrange(0, currentNodeNum)
    nodeA = list(G.nodes)[idx]
    
    # find the nearest node
    nearest = None
    distance = np.inf
    for i in G[nodeA]:
        temp_distance = EuclieanDistance(position[nodeA],position[i])
        if temp_distance < distance:
            distance = temp_distance
            nearest = i
    
    # start merge
    newNode = merge(G, nodeA, nearest)
    # update information
    position[newNode] = sum([np.array(position[n]) for n in newNode.split('/')])/ \
                        len(newNode.split('/'))
'''  
    

"\nG = nx.Graph()\nG.add_edges_from(adjList)\n# in order to make d districts, we need to run merge for n-d times\noriginalNodeNum = len(G.nodes)\nfor i in range(originalNodeNum-DISTRICT_NUM):\n    # randomly choose one node\n    currentNodeNum = originalNodeNum - i\n    idx = random.randrange(0, currentNodeNum)\n    nodeA = list(G.nodes)[idx]\n    \n    # find the nearest node\n    nearest = None\n    distance = np.inf\n    for i in G[nodeA]:\n        temp_distance = EuclieanDistance(position[nodeA],position[i])\n        if temp_distance < distance:\n            distance = temp_distance\n            nearest = i\n    \n    # start merge\n    newNode = merge(G, nodeA, nearest)\n    # update information\n    position[newNode] = sum([np.array(position[n]) for n in newNode.split('/')])/                         len(newNode.split('/'))\n"

In [427]:

G = nx.Graph()
G.add_edges_from(adjList)
# in order to make d districts, we need to run merge for n-d times
originalNodeNum = len(G.nodes)
for i in range(originalNodeNum-DISTRICT_NUM):
    # randomly choose one node
    currentNodeNum = originalNodeNum - i
    idx = random.randrange(0, currentNodeNum)
    nodeA = list(G.nodes)[idx]
    #print(nodeA)
    # find the nearest node
    
    centroidA = [0,0,0] #first two are coordinate, and the last one is number of merged nodes
    
    for j in nodeA.split("/"):
        centroidA[0] += position[j][0]
        centroidA[1] += position[j][1]
        centroidA[2] += 1
        
    centroidA[0] = centroidA[0]/centroidA[2]
    centroidA[1] = centroidA[1]/centroidA[2]
    
    nearest = None
    distance = np.inf
    for i in G[nodeA]:
        temp_distance = EuclieanDistance(centroidA,position[i])
        if temp_distance < distance:
            distance = temp_distance
            nearest = i
    
    # start merge
    newNode = merge(G, nodeA, nearest)
    # update information
    position[newNode] = sum([np.array(position[n]) for n in newNode.split('/')])/ \
                        len(newNode.split('/'))


Start reassigning population

In [428]:
# get initial assign information
district = {}
dNum = 1
for n in G.nodes:
    for i in n.split("/"):
        district[i] = dNum
    dNum += 1

In [429]:
GPop = nx.Graph()
GPop.add_edges_from(adjList)
nx.set_node_attributes(GPop,district,name = "assign")
nx.set_node_attributes(GPop,population,name = "population")

In [430]:
GPop.nodes["n_x3_y0"]

{'assign': 5, 'population': 49}

In [431]:
def findAdj(GPop):
    adjDistricts = []
    adjNodes = []
    for i in GPop.edges:
        nodeADistrict = GPop.nodes[i[0]]["assign"]
        nodeBDistrict = GPop.nodes[i[1]]["assign"]
        if nodeADistrict != nodeBDistrict:
            adjDistricts.append((nodeADistrict,nodeBDistrict))
            adjNodes.append((i,(nodeADistrict,nodeBDistrict)))
    return list(set(adjDistricts)), adjNodes

In [432]:
def findPopulation(GPop):
    popDistrict = {}
    for i in range(1,DISTRICT_NUM+1):
        total = 0
        for n in GPop.nodes:
            if GPop.nodes[n]["assign"] == i:
                total += GPop.nodes[n]["population"]
        popDistrict[i] = total
    return popDistrict


In [433]:
def findPosition(GPop):
    positionDistrict = {}
    for i in range(1,DISTRICT_NUM+1):
        positionDistrict[i] = [0,0,0]
    for i in GPop.nodes:
        positionDistrict[GPop.nodes[i]["assign"]][0] +=  position[i][0]
        positionDistrict[GPop.nodes[i]["assign"]][1] +=  position[i][1]
        positionDistrict[GPop.nodes[i]["assign"]][2] +=  1
    
    for i in range(1,DISTRICT_NUM+1):
        positionDistrict[i] = [positionDistrict[i][0]/positionDistrict[i][2], \
                               positionDistrict[i][1]/positionDistrict[i][2], \
                               positionDistrict[i][2]]
    return positionDistrict

In [434]:
def returnSecond(x):
    return x[1]

In [ ]:
tStart = time.time()
# start population adjustment
lastInfeasible = None
for i in range(1000):
    adjDistricts, adjNodes= findAdj(GPop)
    popDistrict = findPopulation(GPop)
    posDistrict = findPosition(GPop)
    
    largestDiff = 0
    largestAdj = None
    for j in adjDistricts:
        absDiff = abs(popDistrict[j[0]] - popDistrict[j[1]])
        if absDiff > largestDiff:
            largestDiff = absDiff
            largestAdj = j
    
    sourceDistrict = None
    destinationDistrict = None
    
    if popDistrict[largestAdj[0]] > popDistrict[largestAdj[1]]:
        sourceDistrict = largestAdj[0]
        destinationDistrict = largestAdj[1]
    else:
        sourceDistrict = largestAdj[1]
        destinationDistrict = largestAdj[0]
        
    # get all the inner border points
    sourceNodes = [
        node
        for node, data
        in GPop.nodes(data=True)
        if data.get("assign") == sourceDistrict
    ]
    
    borderPoints = []
    for a in adjNodes:
        if sourceDistrict in a[1] and destinationDistrict in a[1]:
            for n in a[0]:
                if n in sourceNodes:
                    borderPoints.append(n)
    # choose the one that maximize the compactness                
    tempDist = []
    for p in borderPoints:     
        tempDist.append((p,
                        EuclieanDistance(position[p],posDistrict[destinationDistrict]) - \
                        EuclieanDistance(position[p],posDistrict[sourceDistrict])))
        
    tempDist = sorted(tempDist,key = returnSecond, reverse=False)
    
    # check contiguity constraint
    chosenNode = None
    for i in tempDist:
        removeNode = i[0]
        subNodes = set(sourceNodes) - set([removeNode])
        tempG = GPop.subgraph(subNodes)
        if nx.is_connected(tempG):
            chosenNode = removeNode
            break
            
    if chosenNode == None:
        print("No node is chosen!!!")
        
    # reassign
    GPop.nodes[chosenNode]["assign"] = destinationDistrict
    
tEnd = time.time()

In [ ]:
tempDist

In [ ]:
assignDict = nx.get_node_attributes(GPop, "assign")
data_list = []
for i in range(ASSIGN_W):
    temp_list = []
    for j in range(ASSIGN_H):
        temp_list.append(assignDict["n_x{}_y{}".format(i,j)])
    data_list.append(temp_list)

data_list.reverse()

In [ ]:
custom_color = []
bounds = []
coloar_patches = []
district_idx = {}
count = 1
for i in districtSet:
    custom_color.append((random.random(),random.random(),random.random()))
    coloar_patches.append(mpatches.Patch(color = custom_color[-1],label = i))
    bounds.append(count)
    district_idx[i] = count
    count += 1

In [ ]:
W = []
for i in range(ASSIGN_W):
    W.append('W'+str(i+1))
H = []
for i in range(ASSIGN_H):
    H.append('H'+str(i+1))

In [ ]:
data = data_list
cmap = colors.ListedColormap(custom_color)
norm = colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
bounds

In [ ]:
figure(figsize=(20, 20), dpi=80)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(data, cmap=cmap, norm=norm)

# draw gridlines
ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
ax.set_xticks(np.arange(0.5, ASSIGN_W+0.5, 1))
ax.set_yticks(np.arange(0.5, ASSIGN_H+0.5, 1))
ax.set_xticklabels(W)
ax.set_yticklabels(H)
ax.set_title('District Visualization')
#plt.legend(handles=coloar_patches)
plt.show()

In [ ]:
assignDict = district
data_list = []
for i in range(ASSIGN_W):
    temp_list = []
    for j in range(ASSIGN_H):
        temp_list.append(assignDict["n_x{}_y{}".format(i,j)])
    data_list.append(temp_list)

data_list.reverse()

data = data_list
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(data, cmap=cmap, norm=norm)

# draw gridlines
ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
ax.set_xticks(np.arange(0.5, ASSIGN_W+0.5, 1))
ax.set_yticks(np.arange(0.5, ASSIGN_H+0.5, 1))
ax.set_xticklabels(W)
ax.set_yticklabels(H)
ax.set_title('District Visualization')
#plt.legend(handles=coloar_patches)
plt.show()

In [ ]:
t2 = time.time()